
## CREATE A SHORTCUT OF THE SHARED FOLDER(the drive link for controlnet provided in github repo) IN YOUR GOOGLE DRIVE. THE CODE SHOULD RUN FINE. USE GPU.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install diffusers

In [ ]:
%cd /content/drive/MyDrive/EVERYTHING_CONTROLNET

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.utils import load_image
import torch
import matplotlib.pyplot as plt
from PIL import Image

controlnet = ControlNetModel.from_pretrained("./controlnet", torch_dtype=torch.float16)
pipeline = StableDiffusionControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1", controlnet=controlnet, torch_dtype=torch.float16
).to("cuda")

### TO CONFIGURE THIS AS BACKEND RUN THE FOLLOWING CELLS

In [ ]:
# Step 1: Install libraries
!pip install flask flask-ngrok
!pip install flask pyngrok

In [ ]:
# Replace 'YOUR_NGROK_AUTH_TOKEN_HERE' with your actual ngrok auth token.
# You can get your token by signing up at https://dashboard.ngrok.com/get-started/setup
!ngrok config add-authtoken YOUR_NGROK_AUTH_TOKEN_HERE


In [ ]:
!pip install flask_cors

In [ ]:
# EACH TIME THIS CELL IS RUN, A UNIQUE NGROK URL IS CREATED, IT NEEDS TO THE COPIED IN THE FRONTEND
# COPY IT AND PASTE THIS AT THE TOP OF THIS FILE IN THE FRONTEND::   frontend/src/pages/Sketch/Index.tsx

from flask import Flask, request, jsonify
from pyngrok import ngrok
from PIL import Image
import io
import random
import base64
import os
import torch
import matplotlib.pyplot as plt
from flask_cors import CORS

app = Flask(__name__)
CORS(app)


@app.route('/')
def home():
    return "Backend is live!"

@app.route('/upload-image', methods=['POST', 'OPTIONS'])
def upload_image():
    if request.method == 'OPTIONS':
        # Handle preflight
        return '', 204

    try:
        data = request.get_json(force=True)
        image_base64 = data.get('image_url')
        # no_of_images= data.get('no_of_image')
        prompt = data.get('prompt')

        if not image_base64 or not prompt:
            return jsonify({'error': 'Missing image or prompt'}), 400

        print("\n--- Incoming Request ---")
        print("Prompt:", prompt)
        print("Received base64 (first 100 chars):", image_base64[:100])
        print("--- End Incoming Request ---\n")

        # Decode the base64 image
        image_data = base64.b64decode(image_base64)
        input_image = Image.open(io.BytesIO(image_data)).convert("RGB")

        # Resize to 512x512
        control_image = input_image.resize((512, 512))

        # Generate images
        generated_images = []
        for i in range(5):
            generator = torch.manual_seed(i)
            result = pipeline(prompt=prompt, image=control_image, num_inference_steps=20, generator=generator)
            generated_images.append(result.images[0])

        # Convert all images to base64
        base64_images = []
        for idx, img in enumerate(generated_images):
            buffer = io.BytesIO()
            img.save(buffer, format="PNG")
            buffer.seek(0)
            img_base64 = base64.b64encode(buffer.read()).decode('utf-8')
            base64_images.append(img_base64)

        print("\n--- Outgoing Response ---")
        print(f"Returning {len(base64_images)} images")
        print("First image base64 (first 100 chars):", base64_images[0][:100])
        print("--- End Outgoing Response ---\n")

        return jsonify({'images_base64': base64_images})

    except Exception as e:
        print("Error occurred:", str(e))
        return jsonify({'error': str(e)}), 500

# Start server with ngrok
public_url = ngrok.connect(5000)
print("Your ngrok URL:", public_url)

app.run(port=5000)

Your ngrok URL: NgrokTunnel: "https://eb13-34-125-127-18.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 05:10:17] "OPTIONS /upload-image HTTP/1.1" 204 -



--- Incoming Request ---
Prompt: dress colourful
Received base64 (first 100 chars): /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAx
--- End Incoming Request ---



  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 05:10:39] "POST /upload-image HTTP/1.1" 200 -



--- Outgoing Response ---
Returning 5 images
First image base64 (first 100 chars): iVBORw0KGgoAAAANSUhEUgAAAgAAAAIACAIAAAB7GkOtAAEAAElEQVR4nGz9WYwsW3YliK299zlm5kOMd773zXPOzIFZHIrFIotV
--- End Outgoing Response ---

